In [65]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import warnings
warnings.filterwarnings('ignore')

In [66]:
players = ["pedro gallese", "carlos zambrano", "miguel araujo", "alexander callens", "andy polo", "marcos lópez", "wilder cartagena",
           "piero quispe", "sergio peña", "edison flores", "gianluca lapadula"]

In [67]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

service = Service('C:/Users/Fabrizio/Documents/Personal Projects/chromedriver.exe')

driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://www.transfermarkt.pe/pedro-gallese/profil/spieler/95172")

wait = WebDriverWait(driver, 5)

players_info = []

def close_privacy_iframe():
    try:
        iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[id^='sp_message_iframe']")))
        driver.switch_to.frame(iframe)
        close_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceptar')]")))
        close_button.click()
        driver.switch_to.default_content()
        time.sleep(5)
    except Exception as e:
        print(f"No se encontró o no se pudo cerrar el iframe de privacidad: {e}")

close_privacy_iframe()

for player in players:
    try:
        input_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "tm-header__input--search-field")))
        button_element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "tm-header__input--search-send")))
        input_element.clear()
        input_element.send_keys(player)
        button_element.click()
        time.sleep(2)

        first_result = wait.until(EC.visibility_of_element_located((By.XPATH, "//tbody/tr[1]/td/table/tbody/tr[1]/td[2]/a")))
        first_result.click()
        time.sleep(2)

        info_table = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='info-table info-table--right-space ']")))

        player_info = {}
        content_labels = [element.text.strip().replace(':', '') for element in info_table.find_elements(By.XPATH, ".//span[@class='info-table__content info-table__content--regular']")]
        content_values = [element.text.strip() for element in info_table.find_elements(By.XPATH, ".//span[@class='info-table__content info-table__content--bold']")]
        
        content_values.insert(7, '')
        content_values.insert(8, '')
        content_values.insert(15, '')
        
        for label, value in zip(content_labels, content_values):
            player_info[label] = value
        
        info_image_of_profile_player = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='modal-trigger']/img[@class='data-header__profile-image']")))
        url_image_of_profile_player = info_image_of_profile_player.get_attribute("src")
        player_info["Image URL"] = url_image_of_profile_player
        
        WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "data-header__box--small"))
)

        info_market_box = driver.find_element(By.CLASS_NAME, "data-header__box--small")
        value_market = info_market_box.find_element(By.CLASS_NAME, "data-header__market-value-wrapper")
        last_value_market = info_market_box.find_element(By.CLASS_NAME, "data-header__last-update")
        
        player_info["Valor de mercado actual"] = value_market.text.split("\n")[0]
        player_info["Último revisión"] = last_value_market.text
        
        
        players_info.append(player_info)
        
        driver.get("https://www.transfermarkt.pe/")
        
    except Exception as e:
        print(f"Hubo un problema con el texto '{player}': {e}")
        
driver.quit()


In [77]:
df = pd.DataFrame(players_info).fillna("")
df = df.drop(["Agente", "Club actual", "Redes sociales"], axis = 1)
df.head()

,Nombre en país de origen,F. Nacim./Edad,Lugar de nacimiento,Altura,Nacionalidad,Posición,Pie,Fichado,Contrato hasta,Opción de contrato,Última renovación,Proveedor,Image URL,Valor de mercado actual,Último revisión,Prestado de,Contrato allí hasta
0,Pedro David Gallese Quiroz,23/02/1990 (34),Lima,"1,89 m",Perú,Portero,izquierdo,17/01/2020,31/12/2024,Un año más por parte del club,27/12/2022,adidas,https://img.a.transfermarkt.technology/portrai...,"1,20 mill. €",Última revisión: 13/06/2024,,
1,Carlos Augusto Zambrano Ochandarte,10/07/1989 (34),Callao,"1,85 m",Perú,Defensa - Defensa central,derecho,08/01/2023,31/12/2024,,,Nike,https://img.a.transfermarkt.technology/portrai...,375 mil €,Última revisión: 28/05/2024,,
2,Miguel Gianpierre Araujo Blanco,24/10/1994 (29),Lima,"1,81 m",Perú,Defensa - Defensa central,derecho,05/07/2023,31/12/2025,Un año más por parte del club,,Nike,https://img.a.transfermarkt.technology/portrai...,850 mil €,Última revisión: 13/06/2024,,
3,Alexander Martín Marquinho Callens Asín,04/05/1992 (32),Callao,"1,86 m",Perú\n España,Defensa - Defensa central,izquierdo,08/09/2023,30/06/2024,Opción de compra,,Nike,https://img.a.transfermarkt.technology/portrai...,"1,20 mill. €",Última revisión: 30/05/2024,Girona FC,30/06/2025
4,Andy Jorman Polo Andrade,29/09/1994 (29),Lima,"1,74 m",Perú,Delantero - Extremo derecho,derecho,21/03/2022,31/12/2025,,09/12/2023,Nike,https://img.a.transfermarkt.technology/portrai...,"1,00 mill. €",Última revisión: 28/05/2024,,


In [74]:
df.columns

Index(['Nombre en país de origen', 'F. Nacim./Edad', 'Lugar de nacimiento',
       'Altura', 'Nacionalidad', 'Posición', 'Pie', 'Agente', 'Club actual',
       'Fichado', 'Contrato hasta', 'Opción de contrato', 'Última renovación',
       'Proveedor', 'Redes sociales', 'Image URL', 'Valor de mercado actual',
       'Último revisión', 'Prestado de', 'Contrato allí hasta'],
      dtype='object')